In [1]:
%load_ext autoreload
%autoreload 2

import torch
import yaml
import sys
from torchmetrics import MetricCollection
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from pytorch_lightning import seed_everything
sys.path.append('fbsource/fbcode/surreal/')
from maploc.data import MapillaryDataModule
from maploc.module import GenericModule
from maploc.utils.viz_2d import plot_images, features_to_RGB, save_plot, add_text
from maploc.utils.viz_localization import likelihood_overlay, plot_pose, plot_dense_rotations
from maploc.osm.viz import Colormap, plot_nodes
from maploc.models.metrics import Location2DError, AngleError
from maploc.models.hough_voting import argmax_xyr, fuse_gps
from maploc.models.refinement import FeaturemetricRefiner, subpixel_refinement
torch.set_grad_enabled(False);
plt.rcParams.update({'figure.max_open_warning': 0})

In [12]:
conf = OmegaConf.load('fbsource/fbcode/surreal/maploc/conf/data/mapillary_v4.yaml')
conf = OmegaConf.merge(conf, OmegaConf.create(yaml.full_load("""
# Comment the next 3 lines to pull data from Manifold
local_dir: "./data/mapillary_dumps_v4/"
dump_dir: ${.local_dir}
scenes:
    - sanfrancisco_soma
    - sanfrancisco_hayes
    - amsterdam
    - berlin
    - lemans
    - montrouge
    - toulouse
    - nantes
    - vilnius
    - avignon
    - helsinki
    - milan
    - paris
    # - brussels
    # - newyork_hoboken
    # - metropolis_train
split: "splits_mly14_from-v2.json"
# split: null
filter_for: null
tiles_filename: tiles.pkl
resize_image: 512

return_gps: true
loading:
    val: {batch_size: 1, num_workers: 0}
    train: ${.val}
random: false
augmentation: {rot90: false, flip: false, image: {apply: false}}
add_map_mask: false
""")))
OmegaConf.resolve(conf)
dataset = MapillaryDataModule(conf)
dataset.prepare_data()
dataset.setup()

In [2]:
conf = OmegaConf.load('fbsource/fbcode/surreal/maploc/conf/data/mapillary.yaml')
conf = OmegaConf.merge(conf, OmegaConf.create(yaml.full_load("""
# Comment the next 3 lines to pull data from Manifold
local_dir: "./data/mapillary_dumps_v2/"
# local_dir: "./devvm1417.cln0.facebook.com/data/mapillary_dumps_v2/"
dump_dir: ${.local_dir}
scenes:
    - sanfrancisco_soma
    - sanfrancisco_hayes
    - amsterdam
    - berlin
    - lemans
    - montrouge
    - toulouse
    - nantes
    - vilnius
    - avignon
    - helsinki
    - milan
    # - newyork_hoboken
    # - metropolis_train
# split: None
resize_image: 512

return_gps: true
loading:
    val: {batch_size: 1, num_workers: 0}
    train: ${.val}
random: false
augmentation: {rot90: false, flip: false, image: {apply: false}}
add_map_mask: false
""")))
OmegaConf.resolve(conf)
dataset = MapillaryDataModule(conf)
dataset.prepare_data()
dataset.setup()

In [29]:
!head data/kitti/2011_09_26/2011_09_26_drive_0005_sync/image_02/timestamps.txt

In [31]:
with open("data/kitti/2011_09_26/2011_09_26_drive_0005_sync/image_02/timestamps.txt", "r") as fid:
    t = [float(l.split(":")[-1]) for l in fid.read().split("\n") if l]

In [28]:
!head data/kitti/test2_files.txt

In [3]:
exper = "bev1-osm2-mly12-n100_vgg16-vgg13_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_bs10-resize256_attn-fix-2-2-128d_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_bs6-resize256_attn-fix-2-2-256d_norm-d8-nrot32"
exper = "bev1-osm2-mly12-n100_vgg16-vgg13-plane_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13-plane_bs9-resize256_norm-d8-nrot64-prior"

exper = "test_bev1-osm2-mly12-n100_vgg16-vgg13-depth-bins17_bs6-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-bins17_bs9-resize256_norm-d8-nrot64"
exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-bins33_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-direct_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-posenc_bs9-resize256_norm-d8-nrot64"

# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-bins17-nobev_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-bins17-zmin10_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_res50-stride1-vgg13_depth-bins33_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_res101-stride1-vgg13_depth-bins33_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-bins33-nonorm_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg19_depth-bins33_bs9-resize256_norm-d8-nrot64"
# exper = "bev1-osm2-mly12-n100_res101-stride1-vgg13_depth-bins33-nonormfix_bs9-resize256_norm-d8-nrot64"

# exper = "bev1-osm2-mly12-n100_res50-stride1-vgg13_depth-bins33_bs9-resize256_d8-nrot64-normvalid"
# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-bins33_bs9-resize256_d8-nrot64-normvalid"

exper = "bev1-osm2-mly12-n100_res101-vgg13_depth-bins33_bs9-resize512_norm-d8-nrot64"

# exper = "bev1-osm2-mly13-n100_vgg16-vgg13_depth-bins33_bs9-resize256_norm-d8-nrot64"

# exper = "bev1-osm2-mly12-n100_vgg16-vgg13_depth-bins33_bs9-resize256_d8-nrot64-normvalid_retrieval-d128"

# Original v2
exper = "bev1-osm2-mly13-n100_res101-vgg16_depth-posenc_bs9-resize512_d8-nrot64-normvalid-prior"  # R101, posenc
exper = "bev1-osm2-mly13-n100_res50-vgg16_depth-bins33-nonorm_bs9-resize512_d8-nrot64-normvalid-prior" # R50, log-scales
# exper = "bev1-osm2-mly13-n100_res101-vgg16_depth-bins33-nonorm_bs9-resize512_d8-nrot64-normvalid-prior" # R101, log-scales (crashed)
# exper = "bev1-osm2-mly13-n100_res101-vgg16_depth-bins25-log_bs9-resize512_d8-nrot64-normvalid-prior" # R101, log-depth --> ugly BEV artifacts

# Add replicate padding
exper = "bev1-osm2-mly13-n100_res101-vgg16_depth-bins33-nonorm_bs9-resize512_d8-nrot64-normvalid-prior-rep" # R101, log-scales
exper = "bev1-osm2-mly13-n100_res101-vgg16_depth-bins25-log_bs9-resize512_d8-nrot64-normvalid-prior-rep" # R101, log-depth --> BEV artifacts but better

# change to VGG13
exper = "bev1-osm2-mly13-n100_res101-vgg13_depth-bins33-nonorm_bs9-resize512_d8-nrot64-normvalid-prior-rep" # R101, log-scales, rep --> nice BEV but artifacts on map features
# TODO exper = "bev1-osm2-mly13-n100_res101-vgg13_depth-bins25-log-nonorm_bs9-resize512_d8-nrot64-normvalid-prior-rep" # R101, log-depth, rep
# TODO exper = "bev1-osm2-mly13-n100_res101-vgg13_depth-bins25-log-nonorm_bs9-resize512_d8-nrot64-normvalid-rep" # R101, log-depth, rep, no prior

# cannonical focal
# exper = "bev1-osm2-mly13-n100_res101-vgg19_depth-posenc_bs9-resize512-f256_d8-nrot64-normvalid-prior"  # R101, VGG19, posenc --> poor val accuracy
# exper = "bev1-osm2-mly13-n100_res101-vgg13_depth-bins25-log-nonorm_bs9-resize512-f256_d8-nrot64-normvalid-prior-rep" # R101, VGG13, log-depth, rep --> still artifacts in BEV weights

# different log-scale range
exper = "bev1-osm2-mly13-n100_res101-vgg19_depth-bins33-nonorm-1to9_bs9-resize512_d8-nrot64-normvalid-prior-rep"  # R101, VGG19, rep --> looks more accurate

# padding reflect
# exper = "bev1-osm2-mly13-n100_res101-vgg19_depth-bins33-nonorm_bs9-resize512_d8-nrot64-normvalid-prior-rep_reflect" # R101, VGG19, log-scale --> does not remove boundary artifacts

# fix prior to post-norm
exper = "bev1-osm2-mly13-n100_res101-vgg19_depth-bins33-nonorm-1to9_bs9-resize512_d8-nrot64-normvalid-priornonorm-rep"  # prior doesn't learn, lots of artifacts

# exper = "basic-osm2-mly13-n100_res101-vgg19_bs9-resize512_d256"  # retrieval

root = "manifold://psarlin/tree/maploc/experiments"
path = f'{root}/{exper}/last-step.ckpt'
print(path)
cfg = {}
if "bev" in exper:
    cfg = {'model': {"num_rotations": 256}}
    cfg = {'model': {"num_rotations": 256, "apply_map_prior": False}}
model = GenericModule.load_from_checkpoint(path, strict=True, find_best=True, cfg=cfg)
model = model.eval()#.cuda()
assert model.cfg.data.resize_image == dataset.cfg.resize_image

In [80]:
cfg2 = OmegaConf.load('fbsource/fbcode/surreal/maploc/conf/bev_depth.yaml')
cfg2.model.norm_depth_scores = False
cfg2.model.normalize_features = False
cfg2.model.depth_parameterization = "bev_depth_log"
cfg2.model.normalize_scores_by_dim = False
cfg2.model.normalize_scores_by_num_valid = True
cfg2.model.padding_matching = "replicate"
cfg2.model.num_scale_bins = 25
# cfg2.model.map_encoder.backbone.decoder_norm = None
# cfg2.model.map_encoder.backbone = "simple"
# cfg2.model.do_joint_retrieval = True
# cfg2.model.map_encoder.backbone.decoder = [128, 128, 128]
cfg2.model.map_encoder.unary_prior = True
cfg2.model.image_encoder = OmegaConf.load('fbsource/fbcode/surreal/maploc/conf/model/image_encoder/vgg_unet.yaml')
# cfg2.model.image_encoder = OmegaConf.load('fbsource/fbcode/surreal/maploc/conf/model/image_encoder/resnet_fpn.yaml')
# cfg2.model.image_encoder.backbone.remove_stride_from_first_conv = False
cfg2.data = dataset.cfg
OmegaConf.resolve(cfg2)
model = GenericModule(cfg2).eval()

In [ ]:
seed_everything(25) # best = 25
loader = dataset.dataloader("val", shuffle=True)
print(exper)
colormap = Colormap()
metrics = MetricCollection(model.model.metrics())
metrics["xy_gps"] = Location2DError("xy_gps", model.cfg.model.pixel_per_meter)
metrics["xy_fused"] = Location2DError("xy_fused", model.cfg.model.pixel_per_meter)
# metrics["xy_subpix"] = Location2DError("xy_subpix", model.cfg.model.pixel_per_meter)
# metrics["xy_refined"] = Location2DError("xy_refined", model.cfg.model.pixel_per_meter)
metrics["yaw_fused"] = AngleError("yaw_fused")
# metrics["yaw_subpix"] = AngleError("yaw_subpix")
# metrics["yaw_refined"] = AngleError("yaw_refined")
metrics = metrics.to(model.device)
# refiner = FeaturemetricRefiner(model.model.projection_bev.grid_xz, ppm=model.cfg.model.pixel_per_meter)

for i, batch in zip(range(10), loader):
    batch = model.transfer_batch_to_device(batch, model.device, i)
    pred = model(batch)
    uv_gps, = pred["xy_gps"] = batch["xy_gps"]
    uvt_fused = argmax_xyr(fuse_gps(pred["log_probs"], uv_gps, model.cfg.model.pixel_per_meter, sigma=15))
    uv_fused, = pred["xy_fused"] = uvt_fused[..., :2]
    yaw_fused, = pred["yaw_fused"] = uvt_fused[..., -1]
    # pred["xy_refined"], pred["yaw_refined"] = refiner(
    #     uv_fused, yaw_fused, pred["map"]["map_features"][0][0], pred["features_bev"][0],
    #     pred["bev"]["confidence"][0], pred["valid_bev"][0])
    # pred["xy_subpix"], pred["yaw_subpix"] = subpixel_refinement(
    #     pred["log_probs"][0], uv_fused, yaw_fused)

    loss = model.model.loss(pred, batch)
    results = metrics(pred, batch)
    results.pop("xy_expectation_error")
    for k in list(results):
        if "recall" in k:
            results.pop(k)
    print(f'{i} {loss["total"].item():.2f}', {k: round(v.item(), 2) for k, v in results.items()})

    scene = batch["scene"][0]
    name = batch["name"][0]
    view = dataset.dumps[scene][batch["sequence"][0]]["views"][name]
    image = batch["image"][0].permute(1, 2, 0)
    rasters = batch["map"][0]
    uv = batch["xy"][0]
    yaw = batch["roll_pitch_yaw"][0, -1].numpy()
    # print(image.shape, batch["camera"][0]._data)

    lp_uvt = pred["log_probs"][0]
    lp_uv = lp_uvt.max(-1).values
    prob = lp_uv.exp()
    feats_map = pred["map"]["map_features"][0][0]
    feats_q = pred["features_bev"][0]
    mask_bev = pred["valid_bev"][0]
    if "log_prior" in pred["map"]:
        prior = pred["map"]["log_prior"][0][0].sigmoid()
    else:
        prior = None
    norm_q = conf_q = torch.norm(feats_q, dim=0)
    if "bev" in pred and "confidence" in pred["bev"]:
        conf_q = pred["bev"]["confidence"][0]
    conf_q = conf_q.masked_fill(~mask_bev, np.nan)
    norm_q = norm_q.masked_fill(~mask_bev, np.nan)

    if 'pixel_scales' in pred:
        kw = dict(pixel_scales=pred['pixel_scales'], camera=batch["camera"])
    elif 'pixel_depths' in pred:
        kw = dict(polar_depths=pred['pixel_depths'])
    else:
        kw = dict(polar_log_depths=pred['pixel_log_depths'])
    image_bev, _, _ = model.model.projection_bev(
        model.model.projection_polar(
            torch.nn.functional.interpolate(batch["image"], pred["features_image"][0].shape[-2:]),**kw)[0],
            None, batch["camera"].scale(1 / model.model.image_encoder.scales[0]))
    image_bev = image_bev[0].permute(1, 2, 0).numpy()
    image_bev = np.concatenate([image_bev, mask_bev[..., None]], -1)

    feats_map_rgb, = features_to_RGB(feats_map.numpy())
    feats_q_rgb, = features_to_RGB(feats_q.numpy(), masks=[mask_bev])
    # feats_map_rgb, feats_q_rgb, = features_to_RGB(feats_map.numpy(), feats_q.numpy(), masks=[None, mask_bev])
    feats_q_rgb = np.concatenate([feats_q_rgb, mask_bev[..., None]], -1)

    text1 = f'error: {results["xy_max_error"]:.1f}m/{results["yaw_max_error"]:.1f}°'
    # text2 = f'{results["xy_gps"]:.1f}/{results["xy_fused"]:.1f}/{results["xy_refined"]:.1f}/{results["xy_subpix"]:.1f}m'
    # text2 = f'{results["xy_gps"]:.1f}/{results["xy_fused"]:.1f}/{results["xy_subpix"]:.1f}m'
    text2 = f'GPS/fused: {results["xy_gps"]:.1f}/{results["xy_fused"]:.1f}m'
    map_viz = colormap.apply(rasters)
    plot_images([image, map_viz, lp_uv, likelihood_overlay(prob.numpy(), map_viz), feats_map_rgb],
                titles=[text1, text2, 'loglikelihood', 'likelihood', 'map features'], dpi=75, cmaps='jet')
    colormap.add_colorbar()
    plot_nodes(1, rasters[2])
    plot_pose([1], uv, yaw, s=1/35, c="forestgreen", w=0.015)
    plot_pose([1], uv_fused, yaw_fused, s=1/35, c="blue", w=0.015)
    plot_pose([1], uv_gps, c="red")
    plot_pose([1], pred["xy_max"][0], pred["yaw_max"][0], s=1/35, c="k", w=0.015)
    plot_dense_rotations(2, lp_uvt.exp(), s=1/15)
    
    # plot_images([pred["scores"][0].max(-1).values, pred["scores_matching"][0].max(-1).values, pred["scores_retrieval"][0]], cmaps="jet")
    # continue
    plot_images([norm_q, conf_q, feats_q_rgb] + ([] if prior is None else [prior]),
                 titles=["BEV norm", "BEV weight", "BEV features"]+([] if prior is None else ['map prior']), dpi=50, cmaps='jet')
    # plot_images([conf_q, norm_q], cmaps="jet", dpi=50)
    # plot_images([features_to_RGB(pred["image"]["feature_maps"][0][0].numpy())[0]])
    continue
    
    if model.model.conf.depth_parameterization == "bev_depth_posenc":
        depth_prob = torch.softmax(pred['pixel_depths'][0], dim=0)
        depth_norm = depth_prob.sum(-1)
        depth_exp = torch.sum(depth_prob * torch.arange(depth_prob.shape[-1]), -1) / depth_norm
        plot_images([image, depth_exp, depth_norm.log()], cmaps="jet", dpi=50, titles=["image", "depth exp", "depth norm"])
    else:
        if 'pixel_scales' in pred:
            scales = pred['pixel_scales'][0]
        elif 'pixel_depths' in pred:
            scales = pred['pixel_depths'][0]
        else:
            scales = pred['pixel_log_depths'][0].flip(-1)
        if not model.model.conf.norm_depth_scores:
            scales = scales.exp()
        scales_score = torch.sum(scales, -1)        
        scales_exp = torch.sum(scales * torch.arange(scales.shape[-1]), -1) / scales_score
        scales_var = torch.sum(scales * (torch.arange(scales.shape[-1]) - scales_exp.unsqueeze(-1))**2, -1) / scales_score
        p = pred['pixel_log_depths'][0].flip(-1).softmax(0)
        # plot_images([image, scales_exp.exp(), scales_var, scales_score.log(), scales.max(-1).values],
        # plot_images([image, scales.argmax(-1), scales.max(-1).values.log(), scales_score.log(), scales.max(-1).values],
        plot_images([image, p.argmax(-1), p.max(-1).values.log(), scales_score.log(), scales.max(-1).values],
                    cmaps="jet", titles=["image", "scale exp", "scale var", "total score", "max"])

print(metrics.compute())

# Retrieval

In [ ]:
seed_everything(25) # best = 25
loader = dataset.dataloader("val", shuffle=True)
print(exper)
colormap = Colormap()
metrics = MetricCollection(model.model.metrics())
metrics["xy_gps"] = Location2DError("xy_gps", dataset.cfg.pixel_per_meter)
metrics["xy_fused"] = Location2DError("xy_fused", dataset.cfg.pixel_per_meter)
metrics = metrics.to(model.device)

for i, batch in zip(range(1), loader):
    batch = model.transfer_batch_to_device(batch, model.device, i)
    pred = model(batch)
    uv_gps, = pred["xy_gps"] = batch["xy_gps"]
    uvt_fused = argmax_xyr(fuse_gps(pred["log_probs"][..., None], uv_gps, dataset.cfg.pixel_per_meter, sigma=15))
    uv_fused, = pred["xy_fused"] = uvt_fused[..., :2]

    loss = model.model.loss(pred, batch)
    results = metrics(pred, batch)
    results.pop("xy_expectation_error")
    for k in list(results):
        if "recall" in k:
            results.pop(k)
    print(f'{i} {loss["total"].item():.2f}', {k: round(v.item(), 2) for k, v in results.items()})

    scene = batch["scene"][0]
    name = batch["name"][0]
    view = dataset.dumps[scene][batch["sequence"][0]]["views"][name]
    image = batch["image"][0].permute(1, 2, 0)
    rasters = batch["map"][0]
    uv = batch["xy"][0]
    yaw = batch["roll_pitch_yaw"][0, -1].numpy()

    lp_uvt = lp_uv = pred["log_probs"][0]
    if lp_uvt.ndim == 3:
        lp_uv = lp_uvt.max(-1).values
    prob = lp_uv.exp()
    feats_map = pred["map"]["map_features"][0][0]

    feats_map_rgb, = features_to_RGB(feats_map.numpy())
    
    text1 = f'error: {results["xy_max_error"]:.1f}m'
    text2 = f'GPS/fused: {results["xy_gps"]:.1f}/{results["xy_fused"]:.1f}m'
    map_viz = colormap.apply(rasters)
    plot_images([image, map_viz, lp_uv, likelihood_overlay(prob.numpy(), map_viz), feats_map_rgb],
                titles=[text1, text2, 'loglikelihood', 'likelihood', 'map features'], dpi=75, cmaps='jet')
    colormap.add_colorbar()
    plot_nodes(1, rasters[2])
    plot_pose([1], uv, yaw, s=1/35, c="forestgreen", w=0.015)
    plot_pose([1], uv_fused, s=1/35, c="blue", w=0.015)
    plot_pose([1], uv_gps, c="red")
    plot_pose([1], pred["xy_max"][0], s=1/35, c="k", w=0.015)
    plot_dense_rotations(2, lp_uvt.exp(), s=1/15)
    
print(metrics.compute())

In [54]:
mask = torch.zeros(rasters.shape[-2:], dtype=torch.bool)
mask[5:-5, 5:-5] = True
plot_images([lp_uv, lp_uv.masked_fill(~mask, -np.inf)], cmaps="hot")